In [ ]:
import git
import shutil
import os
import requests
import pandas as pd
import subprocess

# run this cell to import the necessary packages first


[Generate Requirements file for a course](#generate-requirements-file-for-a-course)  
[Generate Requirements file](#generate-a-requirements-file-for-a-single-repository)  
[Add actions files to single repo](#add-github-actions-files-to-a-single-repository)  
[Add actions files to a course](#add-actions-files-to-a-course)  
[Fix Images in a course](#fix-images-in-repositories-of-an-entire-course)  
[Trigger Actions for a course](#trigger-branch-splitter-action-from-a-csv-of-urls)  
[Trigger Action for a single repository](#remote-trigger-a-github-action)


In [ ]:
def check_contents(repo_location):
    """
    Checks the contents of the repository.

    Args:
        repo_location (str): Full repository URL.

    Returns:
        bool: True if the repository contains 'splitter.py'.
    """
    owner = 'learn-co-curriculum'
    repo_name = repo_location.split('/')[-1]
    api_url = f'https://api.github.com/repos/{owner}/{repo_name}/contents?ref=curriculum'
    print(f'Checking contents in {api_url}')
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Bearer {MY_TOKEN}'}
    response = requests.get(api_url, headers=headers).json()
    list_of_contents = []
    for item in response:
        list_of_contents.append(item['name'])
    return 'splitter.py' in list_of_contents

In [ ]:
def check_branch(repo_location):
    """Checks the repository for a curriculum branch which is necessary for the github actions to function

    Args:
        repo_location (URL): full repository URL

    Returns:
        bool: True if the repository contains a curriculum branch
    """
    owner = 'learn-co-curriculum'
    repo_name = repo_location.split('/')[-1]
    api_url = f'https://api.github.com/repos/{owner}/{repo_name}/branches'
    print(f'Checking branches in {api_url}')
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Bearer {MY_TOKEN}'}
    response = requests.get(api_url, headers=headers).json()
    branch_list = []
    for branch in response:
        branch_list.append(branch['name'])
        
    return branch_list

In [ ]:
def add_files_to_repo(repo_location):
    """
    Adds files to the repository.

    Args:
        repo_location (str): Full repository URL.

    Raises:
        ValueError: If the repository URL is invalid.
    """
    branches = check_branch(repo_location)
        
    if 'curriculum' in branches:
        owner = 'learn-co-curriculum'
        repo_name = repo_location.split('/')[-1]
        api_url = f'https://api.github.com/repos/{owner}/{repo_name}'
        print(f'Working on {api_url}')
        MY_TOKEN = os.getenv('GITHUB_TOKEN')
        headers = {'Authorization': f'Bearer {MY_TOKEN}'}
        response = requests.get(api_url, headers=headers).json()
        repo_path = os.path.join(os.getcwd(), repo_name)
        workflows_path = os.path.join(repo_path, '.github', 'workflows')

        if 'ssh_url' in response.keys():
            working_dir = os.getcwd()
            repo = git.Repo.clone_from(response['ssh_url'], repo_path)
            for branch in branches:
                if branch == 'curriculum':
                    continue
                else:
                    repo.git.checkout(branch)
                    subprocess.run(['git', 'rm', '-r', '--cached', 'dsc-github-actions-files'], cwd=repo_path)
                    repo.git.add('.')
                    try:
                        repo.git.commit(m="removed dsc-github-actions-files")
                        subprocess.run(['rm', '-rf', 'dsc-github-actions-files'], cwd=repo_path)
                        repo.git.push()
                    except git.GitCommandError:
                        continue
            repo.git.checkout('curriculum')
            os.makedirs(workflows_path, exist_ok=True)
            yaml_file = 'branch_split.yml'
            script_file = 'splitter.py'
            yaml_destination = os.path.join(repo_path, '.github', 'workflows', 'branch_split.yml')
            script_destination = os.path.join(repo_path, 'splitter.py')
            shutil.copyfile(yaml_file, yaml_destination)
            shutil.copyfile(script_file, script_destination)
            repo.git.add(".")
            try:
                repo.git.commit(m="Adding files via FIS actions updater script")
                repo.git.push()
                print(f'Added files to {repo_location}.')
            except git.GitCommandError:
                print('No changes made to repo')
                pass
            
            try:
                repo.git.checkout('main')
            except:
                repo.git.checkout('master')
            os.makedirs(workflows_path, exist_ok=True)
            yaml_file = 'branch_split.yml'
            yaml_destination = os.path.join(repo_path, '.github', 'workflows', 'branch_split.yml')
            shutil.copyfile(yaml_file, yaml_destination)
            repo.git.add(".")
            try:
                repo.git.commit(m="Adding files via FIS actions updater script")
                repo.git.push()
                print(f'Added files to {repo_location}.')
            except git.GitCommandError:
                print('No changes made to repo')
                shutil.rmtree(repo_path)
                pass
                
            shutil.rmtree(repo_path) 
            print('Cleaning up directory tree')
            print('Done updating Actions files')
        else:
            print(f'{api_url} is not a valid url')            
    
    else:
        print('This repo has no curriculum branch')



## Run all above cells to load all necessary dependencies


## Add GitHub Actions Files to a _single_ Repository
Replace the URL in the `add_files_to_repo` function with the URL you wish to update with the actions files.

In [ ]:
"Replace the URL below with the URL you wish to add files to"

add_files_to_repo("https://github.com/learn-co-curriculum/dsc-random-forest-pyspark-intro")

## Create a yaml from canvas for a course and a csv of all lessons in the course


In [ ]:
# list of phases is a list of tuples with the phase number and the course number. The phase number will be used to name the files when saved
list_of_phases = [(5, 6953)]

In [ ]:
# this will create a yaml of the entire course content, then extract the github url from each lesson and generate a csv of those url's

for phase, course in list_of_phases:
    os.system(f"github-to-canvas --query {course} > phase_{phase}_canvas.yml")
    os.system(f"github-to-canvas --map phase_{phase}_canvas.yml --urls-only > phase_{phase}_canvas.csv")
    


## Add Actions Files to a Course
> You must first generate the csv with the code above in [Create a yaml](#create-a-yaml-from-canvas-for-a-course-and-a-csv-of-all-lessons-in-the-course)

In [ ]:
# add actions files to repository
df = pd.read_csv(f'phase_5_canvas.csv')
df.columns = ['url']
completed = []
for i, item in df.iterrows():
    add_files_to_repo(item['url'])
df = df.drop(df.index[completed])

## Fix Images in Repositories of an Entire Course
> You must first generate the csv with the code above in [Create a yaml](#create-a-yaml-from-canvas-for-a-course-and-a-csv-of-all-lessons-in-the-course)

In [ ]:
# fix images in repo
df = pd.read_csv(f'phase_4_canvas.csv')
df.columns = ['url']
completed = []
for i, item in df.iterrows():
    images_update = subprocess.run(['python', '/Users/jeffreyhinkle/fis-canvas/fis_canvas.py', '-r', '--fix_images', '--remote_url', item['url'], '--s3_directory', 'data-science/images'], capture_output=True)
    print(images_update.stdout)

df = df.drop(df.index[completed])

# Trigger Branch Splitter Action from a `csv` of url's
* This will trigger the action for an entire course
* If you only wish to trigger a single repo use this: [Trigger single repo action](#remote-trigger-a-github-action)

In [ ]:
# trigger branch splitter action
df = pd.read_csv(f'phase_1_canvas.csv')
df.columns = ['url']
completed = []
for i, item in df.iterrows():
    owner = 'learn-co-curriculum'
    repo_name = item.split('/')[-1]
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    branch_headers = {
        'Authorization': f'Bearer {MY_TOKEN}'
    }
    branch_url = f"""https://api.github.com/repos/{owner}/{repo_name}/branches"""
    branch_response = requests.get(branch_url, branch_headers).json()
    
    branches = [b['name'] for b  in branch_response]
    
    if 'master' in branches:
        branch = 'master'
    else:
        branch = 'main'
    
    owner = 'learn-co-curriculum'
    repo_name = item.split('/')[-1]
    data = {
        "ref": branch
    }

    api_url = f"""https://api.github.com/repos/{owner}/{repo_name}/actions/workflows/branch_split.yml/dispatches?-d"""
    print(f'triggering workflow in {api_url}')
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Bearer {MY_TOKEN}', "Accept": "application/vnd.github+json"}
    response = requests.post(api_url, headers=headers, json=data)
    print(response.status_code)
df = df.drop(df.index[completed])

# Remote Trigger a GitHub Action
Replace `repo_location` with the url of your repository

In [ ]:
# remote trigger a github action
repo_location = 'https://github.com/learn-co-curriculum/dsc-random-forest-pyspark-lab'
owner = 'learn-co-curriculum'
repo_name = repo_location.split('/')[-1]
data = {
    "ref": "main"
}

api_url = f"""https://api.github.com/repos/{owner}/{repo_name}/actions/workflows/branch_split.yml/dispatches?-d"""
print(f'triggering workflow in {api_url}')
MY_TOKEN = os.getenv('GITHUB_TOKEN')
headers = {'Authorization': f'Bearer {MY_TOKEN}', "Accept": "application/vnd.github+json"}
response = requests.post(api_url, headers=headers, json=data)


# Generate Requirements file for a course
Before you begin, create a file called `master.txt` in this repository.
* First you will need to populate 'list_of_phases' with the __phase__ number and canvas __course__ number you are working with.
* Ensure you have modified your default canvas credentials in your `bash_profile` to align with the credentials for the instance you are working with.
* Run the code to [generate the yaml and csv from canvas](#create-a-yaml-from-canvas-for-a-course-and-a-csv-of-all-lessons-in-the-course) and ensure the files have been saved in the same folder you are working from currently. (This should happen by default)
* Run the cell with the code to generate the requirements files. Take note of any output other than _requirements.txt file updated successfully._ The output will be large so you may need to open it in a text editor.

When you are done with a phase, rename `master.txt` to something like `phase_1_requirements.txt` and create a new blank `master.txt` for the next run.

In [ ]:
# before you run the below code, create a file called 'master.txt' in this working directory
import pandas as pd
import requirements
# add requirements file to repository
df = pd.read_csv(f'phase_4_canvas.csv')
df.columns = ['url']
completed = []
for i, item in df.iterrows():
    requirements.generate_requirements_file(item['url'])
df = df.drop(df.index[completed])

## Generate a Requirements file for a single repository
> See the instructions in [Generate Requirements file for a course](#generate-requirements-file-for-a-course)

In [ ]:
# this will generate a requirements file for a single repository
import requirements
repo_location = "{replace with the html link to the repository}"

requirements.generate_requirements_file(repo_location)